In [30]:
# Load packages

import numpy as np 
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go

import json
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, Polygon, MultiPoint

import re

# Input data file directory       
from pathlib import Path

# point path to the folder where shape files are stored
data_folder = Path("D:\")

# read input file for Bangalore Urban
df_awc = pd.read_csv('awc_572.csv')

# strip quotation marks from all columns
df_awc[df_awc.columns] = df_awc.apply(lambda x: x.str.strip())
df_awc[df_awc.columns] = df_awc.apply(lambda x: x.str.strip('\"'))

df_awc['lon'] = df_awc['lon'].astype(float)
df_awc['lat'] = df_awc['lat'].astype(float)

geometry = [Point(xy) for xy in zip(df_awc['lon'] ,df_awc['lat'])]
df_awc = gpd.GeoDataFrame(df_awc, geometry = geometry, crs="EPSG:4326")

# get eb boundaries
file_to_open = data_folder / "Bangalore_EnumerationBlocks/190531_enumerationBlockDataCleaned_py_wgs84_PT_1.shp"
df_eb = gpd.read_file(file_to_open)
df_eb.sort_values('Uniq_Enume', inplace = True)
df_eb['Uniq_Enume'] = df_eb['Uniq_Enume'].replace({'149_102.': '149_102'}, regex=False)

# eb json
gdf = gpd.GeoDataFrame(df_eb)

j = gdf.to_json()
gdf_json = json.loads(j)

# get ward boundaries
df_ward = df_eb[['Wd_No', 'Wd_Name', 'geometry']]
df_ward['WC'] = df_ward['Wd_No']
df_ward = df_ward.dissolve(by='Wd_No')
df_ward['WC'] = df_ward['WC'].astype(int)

# ward json
k = df_ward.to_json()
gdf_ward_json = json.loads(k)
gdf_ward_json

# get ward name
df_awc_geo = gpd.sjoin(df_awc, df_ward, how='left', op='intersects')
df_awc_geo['Wd_Name'] = df_awc_geo['Wd_Name'].fillna('')
df_awc_geo['WC'] = df_awc_geo['WC'].fillna('')

# Read gsl typology polygons
file_to_open = data_folder / "BLR/BLR_Partial_wgs84.shp"
df_typology = gpd.read_file(file_to_open)

# select typology 1, 2, 3
df_typology['DN'] = df_typology['DN'].astype(int)
df_typology = df_typology.sort_values('DN')
df_typology1 = df_typology[df_typology['DN'].isin([1])]
df_typology1['DN'].unique()
df_typology2 = df_typology[df_typology['DN'].isin([2])]
df_typology2['DN'].unique()
df_typology3 = df_typology[df_typology['DN'].isin([3])]
df_typology3['DN'].unique()

# typology 1 json
gdf_typology1 = gpd.GeoDataFrame(df_typology1, crs="EPSG:4326")

j = gdf_typology1.to_json()
gdf_typology1_json = json.loads(j)

# typology 2 json
gdf_typology2 = gpd.GeoDataFrame(df_typology2, crs="EPSG:4326")

j = gdf_typology2.to_json()
gdf_typology2_json = json.loads(j)

# typology 3 json
gdf_typology3 = gpd.GeoDataFrame(df_typology3, crs="EPSG:4326")

j = gdf_typology3.to_json()
gdf_typology3_json = json.loads(j)

# Join with scraped AWC details
df_scraped = pd.read_csv('awc_monitoring_details_final_572.csv') #Bangalore

df_scraped['awc_code'] = df_scraped['awc_code'].astype('str')
df_scraped['awc_code'] = df_scraped['awc_code'].str.strip()
df_awc_geo['awc_code'] = df_awc_geo['awc_code'].str.strip()
df_awc_geo['awc_code'] = df_awc_geo['awc_code'].apply(lambda x: re.sub('[^0-9]','', x) )
df_merged = df_awc_geo.merge(df_scraped, left_on ='awc_code', right_on = 'awc_code', how='right')
df_merged['snp_3_5'] = df_merged['NoBoys3_5SNP'] + df_merged['NoGirls3_5SNP']

df_merged['mod_mal'] = df_merged['BoysMod0_3NS'] + df_merged['GirlsMod0_3NS']
+ df_merged['BoysMod3_5NS'] + df_merged['GirlsMod3_5NS']

df_merged['sev_mal'] = df_merged['BoysSev0_3NS'] + df_merged['GirlsSev0_3NS']
+ df_merged['BoysSev3_5NS'] + df_merged['GirlsSev3_5NS']

df_merged['SNB_NOBoys_SC_36m_72m'] = df_merged['SNB_NOBoys_SC_36m_72m'].fillna(0).astype('int')

df_merged = df_merged.fillna('')

# Plot AWC
text1 = [ 'Ward: ' + str(feat['properties']['Wd_Name']) + str(' ') + str(feat['properties']['WC'])for feat in gdf_ward_json['features'] ] 

text_on_hover=['AWC:' + df_merged['awc_name'][i] 
                  + '<br>' + 'Contact:' + df_merged['AwwNameCont'][i]
                  + '<br>' + 'Code:' + df_merged['awc_code'][i] + ' Alpha:' + df_merged['awc_alpha_code'][i]
                  + '<br>' + 'Addr:' + df_merged['NousePin'][i]
                  + '<br>' + 'Ward:' + df_merged['Wd_Name'][i]             
                  + '<br>' + 'SNP_3_5:' + str(df_merged['snp_3_5'][i])
                  + '<br>' + 'SNB_NOBoys_SC_36m_72m:' + str(df_merged['SNB_NOBoys_SC_36m_72m'][i])               
                  + '<br>' + 'SNP_Women:' + str(df_merged['WomenSNP'][i])
                  + '<br>' + 'Mod_Mal:' + str(df_merged['mod_mal'][i])
                  + '<br>' + 'Sev_Mal:' + str(df_merged['sev_mal'][i])
                  + '<br>' + 'Type:' + df_merged['typeAwc'][i] + str(', ')+ df_merged['AWC'][i] + str(', ')
                                     + df_merged['OwnBuild'][i] + str(', ')
                                     + df_merged['PuccaBuild'][i] 
                  + '<br>' + 'Water:' + df_merged['DrnkWater'][i] + str(' Toilet:')+ df_merged['Toilet'][i] 
                  
                  
                  for i in range(df_merged.shape[0])]

fig = go.FigureWidget(
    data=
    
    [
        go.Choroplethmapbox(geojson=gdf_typology1_json, 
                             locations=[f["id"] for f in gdf_typology1_json["features"]],
                             z=[1] * len(gdf_typology1_json["features"]),
                             hoverinfo='none',
                             colorscale=[[0, 'rgb(215, 227, 110)'], [1.0, 'rgb(215, 227, 110)']],
                             showscale=False)
    ]
    +
    [
        go.Choroplethmapbox(geojson=gdf_typology2_json, 
                             locations=[f["id"] for f in gdf_typology2_json["features"]],
                             z=[1] * len(gdf_typology2_json["features"]),
                             hoverinfo='none',
                             colorscale=[[0, 'rgb(157, 236, 242)'], [1.0, 'rgb(157, 236, 242)']],
                             showscale=False)
    ]
    +
    [
        go.Choroplethmapbox(geojson=gdf_typology3_json, 
                             locations=[f["id"] for f in gdf_typology3_json["features"]],
                             z=[1] * len(gdf_typology3_json["features"]),
                             hoverinfo='none',
                             colorscale=[[0, 'rgb(251, 110, 64)'], [1.0, 'rgb(251, 110, 64)']],
                             showscale=False)
    ]
    +
    [
        go.Choroplethmapbox(geojson=gdf_json, #eb
                                    locations=[f["id"] for f in gdf_json["features"]],
                                    z=[1] * len(gdf_json["features"]),
                                    colorscale=[[0, 'rgb(255,255,255)'], [0.5, 'rgb(255,255,255)'], [1.0, 'rgb(255,255,255)']], 
                                    showscale=False,
                                    text=text1,
                                    hoverinfo='none',
                                    hovertemplate=
                                        "%{text}<br>" ,
                                    marker_opacity=0.2, 
                                    marker_line_width=1,
                                    marker_line_color='brown')
    ]
    +
    [
        go.Choroplethmapbox(geojson=gdf_ward_json, 
                                    locations=[f["id"] for f in gdf_ward_json["features"]],
                                    z=[1] * len(gdf_ward_json["features"]),
                                    colorscale=[[0, 'rgb(255,255,255)'], [0.5, 'rgb(255,255,255)'], [1.0, 'rgb(255,255,255)']], 
                                    showscale=False,
                                    text=text1,
                                    hoverinfo='all',
                                    marker_opacity=0.25, 
                                    marker_line_width=2,
                                    marker_line_color='brown'
                           )
    ]
    +
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='AWC location',
            marker=go.scattermapbox.Marker(symbol='circle', size=8, color='blue', opacity=0.6)
        )
    ]
    +
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='SNP 3 to 5 yrs count',
            visible='legendonly',
            marker=go.scattermapbox.Marker(symbol='circle', size=8, color=df_merged['snp_3_5'], colorscale="viridis_r", opacity=0.8)
        )
    ]
    +
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='SNP 3 to 5 yrs count 90th percentile',
            visible='legendonly',
            marker=go.scattermapbox.Marker(symbol='circle', size=8, color=(df_merged['snp_3_5'] > df_merged[df_merged['snp_3_5'] > 0]['snp_3_5'].quantile(0.9)).astype('int'), colorscale="viridis_r", opacity=0.8)

        )
    ]
    +
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='SNP SC Boys 3 to 6 yrs count',
            visible='legendonly',
            marker=go.scattermapbox.Marker(symbol='circle', size=8, color=df_merged['SNB_NOBoys_SC_36m_72m'], colorscale="viridis_r", opacity=0.8)
        )
    ]
    +
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='Moderately Malnourished 0 to 5 yrs',
            visible='legendonly',
            marker=go.scattermapbox.Marker(symbol='circle', size=8, color=df_merged['mod_mal'], colorscale="viridis_r", opacity=0.8)
        )
    ]
    +
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='Severly Malnourished 0 to 5 yrs',
            visible='legendonly',
            marker=go.scattermapbox.Marker(symbol='circle', size=8, color=df_merged['sev_mal'], colorscale="viridis_r", opacity=0.8)
        )
    ]  
    +
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='SNP women count',
            visible='legendonly',
            marker=go.scattermapbox.Marker(symbol='circle', size=8, color=df_merged['WomenSNP'], colorscale="viridis_r", opacity=0.8)
        )
    ]

)

fig.update_layout(
#                  mapbox_style="open-street-map",
                  mapbox_style="carto-positron",
#                  mapbox_style="white-bg",
#                  mapbox_style="stamen-terrain",
                  mapbox_zoom=11, 
                  mapbox_center = {"lat": 12.99, "lon": 77.59},
                  margin={"r":0,"t":0,"l":0,"b":0},
                  title={
                    'text': 'Aanganwadi Centres in Bangalore Urban Plotted Against Neighbourhood Types', 
                    'y':0.97,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'})

fig.update_layout(
    legend=dict(
        x=1,
        y=.85,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        itemsizing="trace",
        bordercolor="peru",
        borderwidth=2
    )
)


#create html file
fig.write_html('awc_scatter_gsl_type_eb_blr.html', auto_open=True)

#**************End of Code ******************

D:\2020\Anaconda\envs\pysal-workshop\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

